# Chapter 7: 高度なMCMC手法

## 学習目標
- 適応的MCMC手法を習得する
- アンサンブルサンプラーの原理を理解する
- 並列・分散MCMCの概念を理解する
- 変分推論との比較を学ぶ
- ベイズ最適化とMCMCの関係を学ぶ
- 実用的なMCMCライブラリの使用方法を習得する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
from scipy.linalg import cholesky, solve_triangular
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 7.1 適応的MCMC手法

適応的MCMCは、サンプリング中にアルゴリズムのパラメータを自動調整する手法群です。固定パラメータの限界を克服し、より効率的なサンプリングを実現します。

### 主要なアプローチ
- **適応的メトロポリス法（AM）**：提案分布の共分散を適応的に調整
- **遅延棄却適応的メトロポリス（DRAM）**：棄却時に縮小提案を試行
- **適応的独立性サンプラー**：独立提案分布を動的に最適化
- **ロバスト適応的メトロポリス（RAM）**：ロバストな共分散推定

In [ ]:
class AdaptiveMetropolis:
    """
    適応的メトロポリス法（AM）
    Haario et al. (2001) による手法
    """
    
    def __init__(self, log_prob_fn, initial_cov_scale=0.1, adaptation_start=100):
        self.log_prob_fn = log_prob_fn
        self.initial_cov_scale = initial_cov_scale
        self.adaptation_start = adaptation_start
        
    def sample(self, initial_value, n_samples):
        """
        適応的サンプリングの実行
        """
        dim = len(initial_value)
        samples = np.zeros((n_samples, dim))
        
        # 初期設定
        current = initial_value.copy()
        current_log_prob = self.log_prob_fn(current)
        
        # 適応的共分散行列
        cov_matrix = self.initial_cov_scale**2 * np.eye(dim)
        epsilon = 1e-6  # 数値安定性のため
        sd_scale = 2.4**2 / dim  # 最適スケーリング
        
        n_accepted = 0
        
        for i in range(n_samples):
            # 提案分布の更新
            if i >= self.adaptation_start:
                # 経験共分散行列の計算
                sample_mean = np.mean(samples[:i], axis=0)
                sample_cov = np.cov(samples[:i].T, ddof=1)
                
                # Regularization（数値安定性のため）
                if np.any(np.isnan(sample_cov)) or np.any(np.isinf(sample_cov)):
                    cov_matrix = self.initial_cov_scale**2 * np.eye(dim)
                else:
                    cov_matrix = sd_scale * (sample_cov + epsilon * np.eye(dim))
            
            # 提案
            try:
                proposed = np.random.multivariate_normal(current, cov_matrix)
            except np.linalg.LinAlgError:
                # 共分散行列が特異の場合
                proposed = current + np.random.normal(0, self.initial_cov_scale, dim)
            
            proposed_log_prob = self.log_prob_fn(proposed)
            
            # 受理確率
            log_alpha = proposed_log_prob - current_log_prob
            alpha = min(1.0, np.exp(log_alpha))
            
            # 受理/棄却
            if np.random.rand() < alpha:
                current = proposed
                current_log_prob = proposed_log_prob
                n_accepted += 1
            
            samples[i] = current
            
            if (i + 1) % 1000 == 0:
                print(f"Iteration {i+1}/{n_samples}, Acceptance rate: {n_accepted/(i+1):.3f}")
        
        acceptance_rate = n_accepted / n_samples
        return samples, acceptance_rate, cov_matrix

# テスト用の2次元正規分布
def multivariate_normal_log_prob(x, mu, cov):
    """多変量正規分布の対数確率密度"""
    diff = x - mu
    try:
        chol = cholesky(cov, lower=True)
        log_det = 2 * np.sum(np.log(np.diag(chol)))
        solve = solve_triangular(chol, diff, lower=True)
        mahalanobis_sq = np.sum(solve**2)
    except np.linalg.LinAlgError:
        return -np.inf
    
    return -0.5 * (len(mu) * np.log(2 * np.pi) + log_det + mahalanobis_sq)

# パラメータ設定
mu_target = np.array([0.0, 0.0])
cov_target = np.array([[1.0, 0.8], [0.8, 1.0]])

# 適応的メトロポリス法の実行
print("適応的メトロポリス法サンプリング実行中...")
am = AdaptiveMetropolis(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target)
)

am_samples, am_acceptance_rate, final_cov = am.sample(
    initial_value=np.array([0.0, 0.0]),
    n_samples=5000
)

print(f"\n適応的メトロポリス法受理率: {am_acceptance_rate:.3f}")
print(f"\n最終提案共分散行列:")
print(final_cov)
print(f"\n真の共分散行列:")
print(cov_target)

In [ ]:
# 適応的メトロポリス法の結果可視化
def plot_adaptive_results(samples, final_cov, mu_target, cov_target, title="Adaptive Metropolis Results"):
    """
    適応的メトロポリス法結果の可視化
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    burnin = 500
    samples_clean = samples[burnin:]
    
    # 1. トレースプロット
    axes[0, 0].plot(samples[:2000, 0], alpha=0.8, label='X1', linewidth=0.8)
    axes[0, 0].plot(samples[:2000, 1], alpha=0.8, label='X2', linewidth=0.8)
    axes[0, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
    axes[0, 0].set_title('Trace Plot')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Value')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 散布図と真の分布の等高線
    axes[0, 1].scatter(samples_clean[::5, 0], samples_clean[::5, 1], alpha=0.6, s=1)
    
    # 真の分布の等高線
    x1_range = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 50)
    x2_range = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 50)
    X1, X2 = np.meshgrid(x1_range, x2_range)
    pos = np.dstack((X1, X2))
    rv = stats.multivariate_normal(mu_target, cov_target)
    axes[0, 1].contour(X1, X2, rv.pdf(pos), colors='red', alpha=0.8, linewidths=2)
    axes[0, 1].set_title('Samples with True Distribution')
    axes[0, 1].set_xlabel('X1')
    axes[0, 1].set_ylabel('X2')
    axes[0, 1].set_aspect('equal')
    
    # 3. 共分散行列の進化
    n_points = min(1000, len(samples) // 5)
    cov_evolution = []
    
    for i in range(100, n_points, 50):
        if i >= 100:
            sample_cov = np.cov(samples[:i].T, ddof=1)
            if not (np.any(np.isnan(sample_cov)) or np.any(np.isinf(sample_cov))):
                cov_evolution.append(sample_cov[0, 1])  # 共分散要素
    
    if cov_evolution:
        axes[0, 2].plot(range(100, 100 + len(cov_evolution) * 50, 50), cov_evolution, 'b-', alpha=0.8)
        axes[0, 2].axhline(cov_target[0, 1], color='red', linestyle='--', label='True covariance')
        axes[0, 2].set_title('Covariance Evolution')
        axes[0, 2].set_xlabel('Iteration')
        axes[0, 2].set_ylabel('Cov(X1, X2)')
        axes[0, 2].legend()
        axes[0, 2].grid(True, alpha=0.3)
    
    # 4. マージナル分布の比較
    axes[1, 0].hist(samples_clean[:, 0], bins=50, density=True, alpha=0.7, 
                   color='lightblue', label='X1 samples')
    x1_theory = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 100)
    axes[1, 0].plot(x1_theory, stats.norm.pdf(x1_theory, mu_target[0], np.sqrt(cov_target[0, 0])), 
                   'r-', linewidth=2, label='X1 true')
    axes[1, 0].set_title('Marginal Distribution X1')
    axes[1, 0].set_xlabel('X1')
    axes[1, 0].set_ylabel('Density')
    axes[1, 0].legend()
    
    # 5. 自己相関関数
    from statsmodels.tsa.stattools import acf
    lags = min(100, len(samples_clean) // 10)
    autocorr_x1 = acf(samples_clean[:, 0], nlags=lags, fft=True)
    autocorr_x2 = acf(samples_clean[:, 1], nlags=lags, fft=True)
    
    axes[1, 1].plot(autocorr_x1, label='X1', alpha=0.8)
    axes[1, 1].plot(autocorr_x2, label='X2', alpha=0.8)
    axes[1, 1].axhline(0, color='k', linestyle='--', alpha=0.5)
    axes[1, 1].axhline(0.05, color='r', linestyle='--', alpha=0.5)
    axes[1, 1].axhline(-0.05, color='r', linestyle='--', alpha=0.5)
    axes[1, 1].set_title('Autocorrelation Functions')
    axes[1, 1].set_xlabel('Lag')
    axes[1, 1].set_ylabel('ACF')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # 6. 統計サマリー
    axes[1, 2].axis('off')
    sample_mean = np.mean(samples_clean, axis=0)
    sample_cov = np.cov(samples_clean.T)
    
    summary_text = f"Statistical Summary\n\n"
    summary_text += f"True mean: [{mu_target[0]:.3f}, {mu_target[1]:.3f}]\n"
    summary_text += f"Sample mean: [{sample_mean[0]:.3f}, {sample_mean[1]:.3f}]\n\n"
    summary_text += f"True covariance:\n[[{cov_target[0,0]:.3f}, {cov_target[0,1]:.3f}],\n"
    summary_text += f" [{cov_target[1,0]:.3f}, {cov_target[1,1]:.3f}]]\n\n"
    summary_text += f"Sample covariance:\n[[{sample_cov[0,0]:.3f}, {sample_cov[0,1]:.3f}],\n"
    summary_text += f" [{sample_cov[1,0]:.3f}, {sample_cov[1,1]:.3f}]]\n\n"
    summary_text += f"Final proposal cov:\n[[{final_cov[0,0]:.3f}, {final_cov[0,1]:.3f}],\n"
    summary_text += f" [{final_cov[1,0]:.3f}, {final_cov[1,1]:.3f}]]"
    
    axes[1, 2].text(0.1, 0.9, summary_text, transform=axes[1, 2].transAxes, 
                   fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# 結果の可視化
plot_adaptive_results(am_samples, final_cov, mu_target, cov_target)

# 効率の計算
def compute_efficiency_metrics(samples, burnin_frac=0.1):
    """
    効率指標の計算
    """
    burnin = int(len(samples) * burnin_frac)
    clean_samples = samples[burnin:]
    
    # 各次元の自己相関時間を計算
    autocorr_times = []
    eff_sample_sizes = []
    
    for dim in range(clean_samples.shape[1]):
        data = clean_samples[:, dim]
        lags = min(200, len(data) // 4)
        autocorr = acf(data, nlags=lags, fft=True)
        
        # 統合自己相関時間
        tau_int = 1.0
        for lag in range(1, len(autocorr)):
            if autocorr[lag] > 0.01:
                tau_int += 2 * autocorr[lag]
            else:
                break
        
        autocorr_times.append(tau_int)
        eff_sample_sizes.append(len(data) / (2 * tau_int + 1))
    
    return autocorr_times, eff_sample_sizes

am_autocorr, am_eff = compute_efficiency_metrics(am_samples)

print(f"\n=== 適応的メトロポリス法効率指標 ===")
print(f"平均自己相関時間: {np.mean(am_autocorr):.2f}")
print(f"平均有効サンプルサイズ: {np.mean(am_eff):.1f}")
print(f"効率: {np.mean(am_eff)/len(am_samples[500:]):.2%}")

## 7.2 アンサンブルサンプラー

アンサンブルサンプラーは、複数のウォーカー（サンプラー）を同時に動かして効率的にサンプリングを行う手法です。特に高次元問題で威力を発揮します。

### Affine Invariant Ensemble Sampler
- Goodman & Weare (2010) により提案
- アフィン変換に不変な性質
- 複数のウォーカーが相互作用しながらサンプリング
- emceeライブラリで実装されている

In [ ]:
class SimpleEnsembleSampler:
    """
    簡易版Affine Invariant Ensemble Sampler
    
    注：これは教育目的の簡易実装です。
    実際のemceeは最適化されたC実装を含みます。
    """
    
    def __init__(self, log_prob_fn, n_walkers, n_dim):
        self.log_prob_fn = log_prob_fn
        self.n_walkers = n_walkers
        self.n_dim = n_dim
        
        if n_walkers < 2 * n_dim:
            raise ValueError(f"n_walkers ({n_walkers}) must be >= 2 * n_dim ({2 * n_dim})")
    
    def sample(self, initial_positions, n_steps):
        """
        アンサンブルサンプリングの実行
        
        Parameters:
        - initial_positions: 初期位置 (n_walkers, n_dim)
        - n_steps: ステップ数
        
        Returns:
        - samples: サンプル配列 (n_steps, n_walkers, n_dim)
        - acceptance_rates: 各ウォーカーの受理率
        """
        samples = np.zeros((n_steps, self.n_walkers, self.n_dim))
        current_positions = initial_positions.copy()
        current_log_probs = np.array([self.log_prob_fn(pos) for pos in current_positions])
        
        n_accepted = np.zeros(self.n_walkers)
        
        for step in range(n_steps):
            # 各ウォーカーに対して提案を生成
            for k in range(self.n_walkers):
                # 補完ウォーカーをランダム選択（自分以外）
                others = list(range(self.n_walkers))
                others.remove(k)
                j = np.random.choice(others)
                
                # ストレッチファクターの生成
                a = 2.0  # ストレッチパラメータ
                z = ((a - 1.0) * np.random.rand() + 1.0) ** 2 / a
                
                # 提案位置の計算
                proposed = current_positions[j] + z * (current_positions[k] - current_positions[j])
                proposed_log_prob = self.log_prob_fn(proposed)
                
                # 受理確率（次元数を考慮）
                log_alpha = (self.n_dim - 1) * np.log(z) + proposed_log_prob - current_log_probs[k]
                alpha = min(1.0, np.exp(log_alpha))
                
                # 受理/棄却
                if np.random.rand() < alpha:
                    current_positions[k] = proposed
                    current_log_probs[k] = proposed_log_prob
                    n_accepted[k] += 1
            
            samples[step] = current_positions.copy()
            
            if (step + 1) % 1000 == 0:
                avg_acceptance = np.mean(n_accepted) / (step + 1)
                print(f"Step {step+1}/{n_steps}, Average acceptance rate: {avg_acceptance:.3f}")
        
        acceptance_rates = n_accepted / n_steps
        return samples, acceptance_rates

# アンサンブルサンプラーのテスト
print("アンサンブルサンプラー実行中...")

# 初期位置の設定（複数のウォーカー）
n_walkers = 20
n_dim = 2
initial_positions = np.random.normal(0, 1, (n_walkers, n_dim))

ensemble_sampler = SimpleEnsembleSampler(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
    n_walkers=n_walkers,
    n_dim=n_dim
)

ensemble_samples, ensemble_acceptance_rates = ensemble_sampler.sample(
    initial_positions=initial_positions,
    n_steps=3000
)

print(f"\n平均受理率: {np.mean(ensemble_acceptance_rates):.3f}")
print(f"受理率の標準偏差: {np.std(ensemble_acceptance_rates):.3f}")

# ウォーカー間の相関を確認
print(f"\nウォーカーごとの受理率:")
for i, rate in enumerate(ensemble_acceptance_rates):
    print(f"Walker {i:2d}: {rate:.3f}")

# 全ウォーカーのサンプルを結合
ensemble_combined = ensemble_samples.reshape(-1, n_dim)
print(f"\n結合サンプル数: {len(ensemble_combined)}")

# 効率指標の計算
ens_autocorr, ens_eff = compute_efficiency_metrics(ensemble_combined)
print(f"平均自己相関時間: {np.mean(ens_autocorr):.2f}")
print(f"平均有効サンプルサイズ: {np.mean(ens_eff):.1f}")
print(f"効率: {np.mean(ens_eff)/len(ensemble_combined[300:]):.2%}")

In [ ]:
# アンサンブルサンプラーの可視化
def plot_ensemble_results(samples, n_walkers, mu_target, cov_target):
    """
    アンサンブルサンプラー結果の可視化
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    burnin = 300
    samples_clean = samples[burnin:]
    combined_samples = samples_clean.reshape(-1, 2)
    
    # 1. 複数ウォーカーのトレースプロット
    colors = plt.cm.tab10(np.linspace(0, 1, min(10, n_walkers)))
    for i in range(min(10, n_walkers)):  # 最大10個のウォーカーを表示
        axes[0, 0].plot(samples[:1500, i, 0], alpha=0.7, color=colors[i], linewidth=0.5)
    axes[0, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
    axes[0, 0].set_title('Walker Traces (X1)')
    axes[0, 0].set_xlabel('Step')
    axes[0, 0].set_ylabel('X1')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 散布図と真の分布
    axes[0, 1].scatter(combined_samples[::20, 0], combined_samples[::20, 1], 
                      alpha=0.6, s=2, c='blue')
    
    # 真の分布の等高線
    x1_range = np.linspace(combined_samples[:, 0].min(), combined_samples[:, 0].max(), 50)
    x2_range = np.linspace(combined_samples[:, 1].min(), combined_samples[:, 1].max(), 50)
    X1, X2 = np.meshgrid(x1_range, x2_range)
    pos = np.dstack((X1, X2))
    rv = stats.multivariate_normal(mu_target, cov_target)
    axes[0, 1].contour(X1, X2, rv.pdf(pos), colors='red', alpha=0.8, linewidths=2)
    axes[0, 1].set_title('Combined Samples')
    axes[0, 1].set_xlabel('X1')
    axes[0, 1].set_ylabel('X2')
    axes[0, 1].set_aspect('equal')
    
    # 3. ウォーカーの軌跡（2次元）
    for i in range(min(5, n_walkers)):
        walker_traj = samples[:200, i, :]
        axes[0, 2].plot(walker_traj[:, 0], walker_traj[:, 1], 
                       alpha=0.7, linewidth=1, label=f'Walker {i}')
        axes[0, 2].plot(walker_traj[0, 0], walker_traj[0, 1], 'o', markersize=4)
    axes[0, 2].set_title('Walker Trajectories (first 200 steps)')
    axes[0, 2].set_xlabel('X1')
    axes[0, 2].set_ylabel('X2')
    axes[0, 2].legend()
    axes[0, 2].set_aspect('equal')
    
    # 4. マージナル分布
    axes[1, 0].hist(combined_samples[:, 0], bins=50, density=True, alpha=0.7, 
                   color='lightblue', label='X1 samples')
    x1_theory = np.linspace(combined_samples[:, 0].min(), combined_samples[:, 0].max(), 100)
    axes[1, 0].plot(x1_theory, stats.norm.pdf(x1_theory, mu_target[0], np.sqrt(cov_target[0, 0])), 
                   'r-', linewidth=2, label='X1 true')
    axes[1, 0].set_title('Marginal Distribution X1')
    axes[1, 0].set_xlabel('X1')
    axes[1, 0].set_ylabel('Density')
    axes[1, 0].legend()
    
    # 5. 自己相関関数
    from statsmodels.tsa.stattools import acf
    lags = min(100, len(combined_samples) // 10)
    autocorr_x1 = acf(combined_samples[:, 0], nlags=lags, fft=True)
    autocorr_x2 = acf(combined_samples[:, 1], nlags=lags, fft=True)
    
    axes[1, 1].plot(autocorr_x1, label='X1', alpha=0.8)
    axes[1, 1].plot(autocorr_x2, label='X2', alpha=0.8)
    axes[1, 1].axhline(0, color='k', linestyle='--', alpha=0.5)
    axes[1, 1].axhline(0.05, color='r', linestyle='--', alpha=0.5)
    axes[1, 1].set_title('Autocorrelation Functions')
    axes[1, 1].set_xlabel('Lag')
    axes[1, 1].set_ylabel('ACF')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    # 6. 統計サマリー
    axes[1, 2].axis('off')
    sample_mean = np.mean(combined_samples, axis=0)
    sample_cov = np.cov(combined_samples.T)
    
    summary_text = f"Ensemble Sampler Summary\n\n"
    summary_text += f"Number of walkers: {n_walkers}\n"
    summary_text += f"Total samples: {len(combined_samples)}\n\n"
    summary_text += f"True mean: [{mu_target[0]:.3f}, {mu_target[1]:.3f}]\n"
    summary_text += f"Sample mean: [{sample_mean[0]:.3f}, {sample_mean[1]:.3f}]\n\n"
    summary_text += f"True covariance:\n[[{cov_target[0,0]:.3f}, {cov_target[0,1]:.3f}],\n"
    summary_text += f" [{cov_target[1,0]:.3f}, {cov_target[1,1]:.3f}]]\n\n"
    summary_text += f"Sample covariance:\n[[{sample_cov[0,0]:.3f}, {sample_cov[0,1]:.3f}],\n"
    summary_text += f" [{sample_cov[1,0]:.3f}, {sample_cov[1,1]:.3f}]]"
    
    axes[1, 2].text(0.1, 0.9, summary_text, transform=axes[1, 2].transAxes, 
                   fontsize=10, verticalalignment='top', fontfamily='monospace')
    
    plt.suptitle('Ensemble Sampler Results', fontsize=16)
    plt.tight_layout()
    plt.show()

# 結果の可視化
plot_ensemble_results(ensemble_samples, n_walkers, mu_target, cov_target)

# ランダムウォークMHとの比較
def random_walk_mh(log_prob_fn, initial_value, n_samples, step_size=0.5):
    """
    ランダムウォーク・メトロポリス・ヘイスティングス法（比較用）
    """
    dim = len(initial_value)
    samples = np.zeros((n_samples, dim))
    current = initial_value.copy()
    current_log_prob = log_prob_fn(current)
    n_accepted = 0
    
    proposal_cov = step_size**2 * np.eye(dim)
    
    for i in range(n_samples):
        # 提案
        proposed = np.random.multivariate_normal(current, proposal_cov)
        proposed_log_prob = log_prob_fn(proposed)
        
        # 受理確率
        log_alpha = proposed_log_prob - current_log_prob
        alpha = min(1.0, np.exp(log_alpha))
        
        # 受理/棄却
        if np.random.rand() < alpha:
            current = proposed
            current_log_prob = proposed_log_prob
            n_accepted += 1
        
        samples[i] = current
    
    acceptance_rate = n_accepted / n_samples
    return samples, acceptance_rate

# 比較のためのランダムウォークMH
print("\n比較用ランダムウォークMH実行中...")
rwmh_samples, rwmh_acceptance_rate = random_walk_mh(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
    initial_value=np.array([0.0, 0.0]),
    n_samples=len(ensemble_combined),
    step_size=0.8
)

rwmh_autocorr, rwmh_eff = compute_efficiency_metrics(rwmh_samples)

print(f"\n=== 手法比較 ===")
print(f"{'Method':<20} {'Acceptance':<12} {'Mean ESS':<12} {'Mean τ':<12} {'Efficiency':<12}")
print("-" * 80)

methods = ['Random Walk MH', 'Adaptive Metropolis', 'Ensemble Sampler']
all_samples = [rwmh_samples, am_samples, ensemble_combined]
all_acceptance = [rwmh_acceptance_rate, am_acceptance_rate, np.mean(ensemble_acceptance_rates)]
all_eff = [rwmh_eff, am_eff, ens_eff]
all_autocorr = [rwmh_autocorr, am_autocorr, ens_autocorr]

for method, acc_rate, eff, autocorr, samples in zip(methods, all_acceptance, all_eff, all_autocorr, all_samples):
    efficiency = np.mean(eff) / len(samples[300:])
    print(f"{method:<20} {acc_rate:<12.3f} {np.mean(eff):<12.1f} {np.mean(autocorr):<12.2f} {efficiency:<12.3f}")

print(f"\n=== 相対性能（Random Walk MH基準） ===")
base_eff = np.mean(rwmh_eff)
for method, eff in zip(methods, all_eff):
    eff_ratio = np.mean(eff) / base_eff
    print(f"{method}: ESS improvement = {eff_ratio:.2f}x")

## 7.3 並列・分散MCMC

大規模データやパラメータ数が多い問題では、並列・分散MCMCが有効です。複数のチェーンを並列実行し、効率的にサンプリングを行います。

### 主要なアプローチ
- **独立並列チェーン**：複数の独立なチェーンを並列実行
- **分散データMCMC**：データを分割して各チェーンで処理
- **Consensus Monte Carlo**：各チェーンの結果を統合
- **Embarrassingly Parallel MCMC**：問題を分割して並列処理

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

def single_chain_runner(args):
    """
    単一チェーン実行のためのワーカー関数
    """
    chain_id, log_prob_fn_params, initial_value, n_samples, step_size, seed = args
    
    # 各プロセスで独立した乱数シードを設定
    np.random.seed(seed)
    
    # 対数確率関数の再構築（プロセス間通信のため）
    mu, cov = log_prob_fn_params
    log_prob_fn = lambda x: multivariate_normal_log_prob(x, mu, cov)
    
    # ランダムウォークMHの実行
    samples, acceptance_rate = random_walk_mh(log_prob_fn, initial_value, n_samples, step_size)
    
    return chain_id, samples, acceptance_rate

def parallel_mcmc_chains(log_prob_fn_params, n_chains=4, n_samples=2000, step_size=0.8):
    """
    複数チェーンの並列実行
    
    Parameters:
    - log_prob_fn_params: 対数確率関数のパラメータ（プロセス間通信用）
    - n_chains: チェーン数
    - n_samples: チェーンあたりのサンプル数
    - step_size: ステップサイズ
    
    Returns:
    - all_samples: 全チェーンのサンプル
    - acceptance_rates: 各チェーンの受理率
    """
    
    # 各チェーンの初期値（互いに異なる）
    np.random.seed(42)
    dim = 2
    initial_values = [np.random.normal(0, 2, dim) for _ in range(n_chains)]
    
    # 各チェーンの引数準備
    chain_args = []
    for i in range(n_chains):
        seed = 42 + i * 1000  # 各チェーンで異なるシード
        chain_args.append((i, log_prob_fn_params, initial_values[i], n_samples, step_size, seed))
    
    print(f"並列MCMC実行中... ({n_chains} chains, {n_samples} samples each)")
    
    # 並列実行
    with ProcessPoolExecutor(max_workers=min(n_chains, mp.cpu_count())) as executor:
        results = list(executor.map(single_chain_runner, chain_args))
    
    # 結果の整理
    all_samples = []
    acceptance_rates = []
    
    for chain_id, samples, acc_rate in sorted(results):
        all_samples.append(samples)
        acceptance_rates.append(acc_rate)
        print(f"Chain {chain_id}: Acceptance rate = {acc_rate:.3f}")
    
    return np.array(all_samples), np.array(acceptance_rates)

def gelman_rubin_diagnostic(chains):
    """
    Gelman-Rubin診断（R-hat統計量）の計算
    
    Parameters:
    - chains: shape (n_chains, n_samples, n_params)
    
    Returns:
    - r_hat: 各パラメータのR-hat値
    """
    n_chains, n_samples, n_params = chains.shape
    
    # 各チェーンから後半のサンプルを使用
    burnin = n_samples // 2
    chains_clean = chains[:, burnin:, :]
    n_samples_clean = chains_clean.shape[1]
    
    r_hat_values = []
    
    for param in range(n_params):
        # 各チェーンの平均と分散
        chain_means = np.mean(chains_clean[:, :, param], axis=1)
        chain_vars = np.var(chains_clean[:, :, param], axis=1, ddof=1)
        
        # 全体平均
        overall_mean = np.mean(chain_means)
        
        # Between-chain variance
        B = n_samples_clean * np.var(chain_means, ddof=1)
        
        # Within-chain variance
        W = np.mean(chain_vars)
        
        # Marginal posterior variance
        var_hat = ((n_samples_clean - 1) * W + B) / n_samples_clean
        
        # R-hat統計量
        if W > 0:
            r_hat = np.sqrt(var_hat / W)
        else:
            r_hat = np.inf
            
        r_hat_values.append(r_hat)
    
    return np.array(r_hat_values)

# 並列MCMCの実行
print("=== 並列MCMC実験 ===")
log_prob_params = (mu_target, cov_target)  # プロセス間通信用のパラメータ

parallel_samples, parallel_acceptance_rates = parallel_mcmc_chains(
    log_prob_fn_params=log_prob_params,
    n_chains=4,
    n_samples=5000,
    step_size=0.8
)

print(f"\n平均受理率: {np.mean(parallel_acceptance_rates):.3f}")
print(f"受理率の標準偏差: {np.std(parallel_acceptance_rates):.3f}")

# Gelman-Rubin診断
r_hat = gelman_rubin_diagnostic(parallel_samples)
print(f"\nGelman-Rubin診断 (R-hat):")
for i, r in enumerate(r_hat):
    status = "✓" if r < 1.1 else "⚠" if r < 1.2 else "✗"
    print(f"  Parameter {i+1}: {r:.4f} {status}")

convergence_status = "収束" if np.all(r_hat < 1.1) else "未収束" if np.any(r_hat > 1.2) else "境界"
print(f"収束判定: {convergence_status}")

# 全チェーンの結合
combined_parallel_samples = parallel_samples.reshape(-1, 2)
print(f"\n結合サンプル数: {len(combined_parallel_samples)}")

# 効率計算
parallel_autocorr, parallel_eff = compute_efficiency_metrics(combined_parallel_samples)
print(f"平均自己相関時間: {np.mean(parallel_autocorr):.2f}")
print(f"平均有効サンプルサイズ: {np.mean(parallel_eff):.1f}")
print(f"効率: {np.mean(parallel_eff)/len(combined_parallel_samples[500:]):.2%}")

## 7.4 変分推論との比較

変分推論（Variational Inference, VI）は、MCMCの代替手法として注目されています。近似的ですが高速な推論が可能です。

### 変分推論の基本概念
- **変分族**：近似事後分布の候補集合
- **ELBO**：Evidence Lower Bound（最大化目標）
- **KLダイバージェンス**：真の事後分布との距離
- **平均場近似**：独立性仮定による簡単化

In [ ]:
class SimpleVariationalInference:
    """
    簡易版変分推論（平均場近似）
    
    仮定：近似事後分布は多変量正規分布
    """
    
    def __init__(self, log_prob_fn, grad_log_prob_fn):
        self.log_prob_fn = log_prob_fn
        self.grad_log_prob_fn = grad_log_prob_fn
    
    def fit(self, initial_mu, initial_log_sigma, n_iterations=2000, learning_rate=0.01, n_samples=100):
        """
        変分パラメータの最適化
        
        Parameters:
        - initial_mu: 平均の初期値
        - initial_log_sigma: 対数標準偏差の初期値
        - n_iterations: 最適化イテレーション数
        - learning_rate: 学習率
        - n_samples: モンテカルロサンプル数（ELBO推定用）
        
        Returns:
        - mu: 最適化された平均
        - sigma: 最適化された標準偏差
        - elbo_history: ELBO値の履歴
        """
        
        # 変分パラメータ
        mu = initial_mu.copy()
        log_sigma = initial_log_sigma.copy()
        
        elbo_history = []
        
        for iteration in range(n_iterations):
            # 現在のパラメータから ELBO とその勾配を計算
            elbo_val, grad_mu, grad_log_sigma = self._compute_elbo_and_gradients(
                mu, log_sigma, n_samples
            )
            
            # パラメータ更新（勾配上昇法）
            mu += learning_rate * grad_mu
            log_sigma += learning_rate * grad_log_sigma
            
            elbo_history.append(elbo_val)
            
            if (iteration + 1) % 500 == 0:
                print(f"Iteration {iteration+1}/{n_iterations}, ELBO: {elbo_val:.4f}")
        
        sigma = np.exp(log_sigma)
        return mu, sigma, np.array(elbo_history)
    
    def _compute_elbo_and_gradients(self, mu, log_sigma, n_samples):
        """
        ELBO とその勾配のモンテカルロ推定
        """
        sigma = np.exp(log_sigma)
        
        # q(θ) からのサンプリング
        epsilon = np.random.normal(0, 1, (n_samples, len(mu)))
        theta_samples = mu + sigma * epsilon
        
        # 各項の計算
        log_p_values = np.array([self.log_prob_fn(theta) for theta in theta_samples])
        log_q_values = -0.5 * np.sum(epsilon**2, axis=1) - 0.5 * len(mu) * np.log(2 * np.pi) - np.sum(log_sigma)
        
        # ELBO = E_q[log p(θ)] - E_q[log q(θ)]
        elbo = np.mean(log_p_values - log_q_values)
        
        # 勾配の計算（再パラメータ化勾配）
        grad_mu = np.mean([self.grad_log_prob_fn(theta) for theta in theta_samples], axis=0)
        
        # log_sigma に対する勾配
        grad_log_sigma = np.zeros_like(log_sigma)
        for i in range(len(log_sigma)):
            # ∂ELBO/∂log_σ_i = E[∂log p(θ)/∂θ_i * ε_i] + 1
            grad_contribution = np.mean([
                self.grad_log_prob_fn(theta)[i] * epsilon[j, i] 
                for j, theta in enumerate(theta_samples)
            ])
            grad_log_sigma[i] = grad_contribution + 1  # エントロピー項の寄与
        
        return elbo, grad_mu, grad_log_sigma
    
    def sample(self, mu, sigma, n_samples):
        """
        近似事後分布からのサンプリング
        """
        return np.random.multivariate_normal(mu, np.diag(sigma**2), n_samples)

# 変分推論の実行
print("=== 変分推論実験 ===")
vi = SimpleVariationalInference(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
    grad_log_prob_fn=lambda x: multivariate_normal_grad_log_prob(x, mu_target, cov_target)
)

# 初期値の設定
initial_mu_vi = np.array([1.0, 1.0])  # 真の値から少しずらす
initial_log_sigma_vi = np.array([0.0, 0.0])  # log(1) = 0

print("変分推論最適化実行中...")
vi_mu, vi_sigma, elbo_history = vi.fit(
    initial_mu=initial_mu_vi,
    initial_log_sigma=initial_log_sigma_vi,
    n_iterations=2000,
    learning_rate=0.01,
    n_samples=100
)

print(f"\n変分推論結果:")
print(f"推定平均: [{vi_mu[0]:.3f}, {vi_mu[1]:.3f}]")
print(f"推定標準偏差: [{vi_sigma[0]:.3f}, {vi_sigma[1]:.3f}]")
print(f"真の平均: [{mu_target[0]:.3f}, {mu_target[1]:.3f}]")
print(f"真の標準偏差: [{np.sqrt(cov_target[0,0]):.3f}, {np.sqrt(cov_target[1,1]):.3f}]")

# 変分推論からのサンプリング
vi_samples = vi.sample(vi_mu, vi_sigma, 5000)
print(f"変分推論サンプル数: {len(vi_samples)}")

# 効率の評価（変分推論では相関がないため簡単）
vi_sample_mean = np.mean(vi_samples, axis=0)
vi_sample_cov = np.cov(vi_samples.T)

print(f"\nサンプル統計:")
print(f"サンプル平均: [{vi_sample_mean[0]:.3f}, {vi_sample_mean[1]:.3f}]")
print(f"サンプル共分散: [{vi_sample_cov[0,0]:.3f}, {vi_sample_cov[0,1]:.3f}]")
print(f"                [{vi_sample_cov[1,0]:.3f}, {vi_sample_cov[1,1]:.3f}]")

# 真の共分散との比較
print(f"\n真の共分散との差:")
cov_error = np.abs(vi_sample_cov - cov_target)
print(f"共分散誤差: [{cov_error[0,0]:.3f}, {cov_error[0,1]:.3f}]")
print(f"            [{cov_error[1,0]:.3f}, {cov_error[1,1]:.3f}]")

In [ ]:
# MCMC手法と変分推論の包括的比較
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# 準備：各手法のサンプルデータ
methods = ['Random Walk MH', 'Adaptive Metropolis', 'Ensemble Sampler', 'Parallel MCMC', 'Variational Inference']
all_samples_comparison = [rwmh_samples, am_samples, ensemble_combined, combined_parallel_samples, vi_samples]
colors = ['red', 'orange', 'blue', 'green', 'purple']

# 1. トレースプロット比較（MCMC手法のみ）
mcmc_methods = methods[:4]
mcmc_samples = all_samples_comparison[:4]
mcmc_colors = colors[:4]

for i, (method, samples, color) in enumerate(zip(mcmc_methods, mcmc_samples, mcmc_colors)):
    axes[0, 0].plot(samples[:1500, 0], alpha=0.7, label=method, color=color, linewidth=0.8)
axes[0, 0].axvline(300, color='gray', linestyle='--', alpha=0.7, label='Burn-in')
axes[0, 0].set_title('Trace Plot Comparison (X1)')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('X1')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 散布図比較（全手法）
for i, (method, samples, color) in enumerate(zip(methods, all_samples_comparison, colors)):
    burnin = 300 if method != 'Variational Inference' else 0
    clean_samples = samples[burnin:]
    axes[0, 1].scatter(clean_samples[::20, 0], clean_samples[::20, 1], 
                      alpha=0.6, s=5, label=method, color=color)

# 真の分布の等高線
x1_range = np.linspace(-4, 4, 50)
x2_range = np.linspace(-4, 4, 50)
X1, X2 = np.meshgrid(x1_range, x2_range)
pos = np.dstack((X1, X2))
rv = stats.multivariate_normal(mu_target, cov_target)
axes[0, 1].contour(X1, X2, rv.pdf(pos), colors='black', alpha=0.8, linewidths=2)
axes[0, 1].set_title('Sample Scatter Plot Comparison')
axes[0, 1].set_xlabel('X1')
axes[0, 1].set_ylabel('X2')
axes[0, 1].legend()
axes[0, 1].set_aspect('equal')

# 3. ELBO収束プロット（変分推論）
axes[0, 2].plot(elbo_history, color='purple', alpha=0.8)
axes[0, 2].set_title('Variational Inference: ELBO Convergence')
axes[0, 2].set_xlabel('Iteration')
axes[0, 2].set_ylabel('ELBO')
axes[0, 2].grid(True, alpha=0.3)

# 4. 自己相関関数比較（MCMC手法のみ）
for i, (method, samples, color) in enumerate(zip(mcmc_methods, mcmc_samples, mcmc_colors)):
    burnin = 300
    clean_samples = samples[burnin:]
    if len(clean_samples) > 100:
        lags = min(50, len(clean_samples) // 10)
        autocorr = acf(clean_samples[:, 0], nlags=lags, fft=True)
        axes[1, 0].plot(autocorr, label=method, alpha=0.8, color=color)

axes[1, 0].axhline(0, color='k', linestyle='--', alpha=0.5)
axes[1, 0].axhline(0.05, color='gray', linestyle='--', alpha=0.5)
axes[1, 0].set_title('Autocorrelation Comparison (X1)')
axes[1, 0].set_xlabel('Lag')
axes[1, 0].set_ylabel('ACF')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 5. マージナル分布比較
x1_range_plot = np.linspace(-4, 4, 100)
true_marginal = stats.norm.pdf(x1_range_plot, mu_target[0], np.sqrt(cov_target[0, 0]))
axes[1, 1].plot(x1_range_plot, true_marginal, 'k-', linewidth=3, label='True', alpha=0.8)

for i, (method, samples, color) in enumerate(zip(methods, all_samples_comparison, colors)):
    burnin = 300 if method != 'Variational Inference' else 0
    clean_samples = samples[burnin:]
    axes[1, 1].hist(clean_samples[:, 0], bins=30, density=True, alpha=0.5, 
                   color=color, label=method)

axes[1, 1].set_title('Marginal Distribution X1')
axes[1, 1].set_xlabel('X1')
axes[1, 1].set_ylabel('Density')
axes[1, 1].legend()

# 6. 効率指標比較
method_names_short = ['RWMH', 'AM', 'Ensemble', 'Parallel', 'VI']
all_eff_comparison = []

for i, samples in enumerate(all_samples_comparison):
    if i < 4:  # MCMC手法
        autocorr, eff = compute_efficiency_metrics(samples)
        all_eff_comparison.append(np.mean(eff))
    else:  # 変分推論
        all_eff_comparison.append(len(samples))  # 全サンプルが独立

axes[1, 2].bar(method_names_short, all_eff_comparison, alpha=0.7, color=colors)
axes[1, 2].set_title('Effective Sample Size Comparison')
axes[1, 2].set_ylabel('Effective Sample Size')
axes[1, 2].tick_params(axis='x', rotation=45)
axes[1, 2].grid(True, alpha=0.3)

# 7. 計算時間の概念的比較（実際の測定ではなく理論的）
# 注：実際の実装では時間計測を行う
computational_cost = [1.0, 1.2, 2.5, 0.8, 0.1]  # 相対的コスト（概念的）
efficiency_per_cost = [eff/cost for eff, cost in zip(all_eff_comparison, computational_cost)]

axes[2, 0].bar(method_names_short, computational_cost, alpha=0.7, color=colors)
axes[2, 0].set_title('Relative Computational Cost')
axes[2, 0].set_ylabel('Relative Cost')
axes[2, 0].tick_params(axis='x', rotation=45)
axes[2, 0].grid(True, alpha=0.3)

# 8. 効率・コスト比
axes[2, 1].bar(method_names_short, efficiency_per_cost, alpha=0.7, color=colors)
axes[2, 1].set_title('Efficiency per Unit Cost')
axes[2, 1].set_ylabel('ESS per Unit Cost')
axes[2, 1].tick_params(axis='x', rotation=45)
axes[2, 1].grid(True, alpha=0.3)

# 9. 統計サマリー
axes[2, 2].axis('off')
summary_text = "Method Comparison Summary\n\n"

# 統計の計算
for i, (method, samples) in enumerate(zip(methods, all_samples_comparison)):
    burnin = 300 if method != 'Variational Inference' else 0
    clean_samples = samples[burnin:]
    sample_mean = np.mean(clean_samples, axis=0)
    
    if i < 4:  # MCMC手法
        autocorr, eff = compute_efficiency_metrics(samples)
        mean_eff = np.mean(eff)
        mean_tau = np.mean(autocorr)
        summary_text += f"{method}:\n"
        summary_text += f"  Mean: [{sample_mean[0]:.3f}, {sample_mean[1]:.3f}]\n"
        summary_text += f"  ESS: {mean_eff:.1f}, τ: {mean_tau:.2f}\n\n"
    else:  # 変分推論
        summary_text += f"{method}:\n"
        summary_text += f"  Mean: [{sample_mean[0]:.3f}, {sample_mean[1]:.3f}]\n"
        summary_text += f"  Independent samples: {len(clean_samples)}\n\n"

axes[2, 2].text(0.1, 0.9, summary_text, transform=axes[2, 2].transAxes, 
               fontsize=9, verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

# 定量的比較表
print(f"\n=== 全手法の定量的比較 ===")
print(f"{'Method':<20} {'Sample Mean X1':<15} {'Sample Mean X2':<15} {'Mean ESS':<12} {'Notes':<20}")
print("-" * 90)

for i, (method, samples) in enumerate(zip(methods, all_samples_comparison)):
    burnin = 300 if method != 'Variational Inference' else 0
    clean_samples = samples[burnin:]
    sample_mean = np.mean(clean_samples, axis=0)
    
    if i < 4:  # MCMC手法
        autocorr, eff = compute_efficiency_metrics(samples)
        mean_eff = np.mean(eff)
        notes = f"τ={np.mean(autocorr):.2f}"
    else:  # 変分推論
        mean_eff = len(clean_samples)
        notes = "Independent samples"
    
    print(f"{method:<20} {sample_mean[0]:<15.3f} {sample_mean[1]:<15.3f} {mean_eff:<12.1f} {notes:<20}")

print(f"\n真の平均: [{mu_target[0]:.3f}, {mu_target[1]:.3f}]")

# 手法選択の指針
print(f"\n=== 手法選択の指針 ===")
selection_guide = {
    "高精度が必要": "Adaptive Metropolis, Ensemble Sampler - 真の分布により近い近似",
    "高速計算が重要": "Variational Inference - 近似的だが非常に高速",
    "並列処理可能": "Parallel MCMC, Ensemble Sampler - マルチコア活用",
    "実装の簡単さ": "Random Walk MH - 理解・実装が容易",
    "高次元問題": "Ensemble Sampler, Adaptive methods - スケーラビリティ",
    "事後分布の探索": "MCMC methods - 分布の詳細な構造を探索",
    "点推定で十分": "Variational Inference - 平均・分散の高速推定"
}

for criteria, recommendation in selection_guide.items():
    print(f"{criteria}: {recommendation}")

print(f"\n=== MCMC vs 変分推論 ===")
comparison_table = {
    "精度": "MCMC > VI (近似誤差なし vs 近似誤差あり)",
    "速度": "VI >> MCMC (最適化 vs サンプリング)",
    "メモリ使用量": "VI < MCMC (パラメータのみ vs 全サンプル)",
    "診断の容易さ": "MCMC > VI (収束診断豊富 vs ELBO監視)",
    "不確実性定量化": "MCMC > VI (完全な分布 vs パラメトリック近似)",
    "実装の複雑さ": "MCMC ≈ VI (どちらも非自明)"
}

for aspect, comparison in comparison_table.items():
    print(f"{aspect}: {comparison}")

## 7.5 実用的なMCMCライブラリの紹介

実際の研究・開発では、専用ライブラリを使用することが一般的です。主要なライブラリを紹介します。

In [ ]:
# 主要なMCMCライブラリの概要と使用例
print("=== 主要なMCMCライブラリ ===")
print()

libraries_info = {
    "PyMC": {
        "description": "Pythonで最も人気のあるベイズ統計ライブラリ",
        "features": [
            "直感的なモデル記述",
            "自動微分によるHMC/NUTS",
            "豊富な分布ライブラリ",
            "変分推論サポート",
            "優れた可視化機能"
        ],
        "use_cases": "階層モデル、時系列分析、機械学習"
    },
    "Stan (PyStan)": {
        "description": "高性能なベイズ推論プラットフォーム",
        "features": [
            "専用言語による高速実行",
            "最先端のNUTS実装",
            "自動微分と最適化",
            "R, Python, Julia等の多言語サポート",
            "詳細な診断機能"
        ],
        "use_cases": "複雑なモデル、大規模データ、研究用途"
    },
    "TensorFlow Probability": {
        "description": "TensorFlowベースの確率的プログラミング",
        "features": [
            "GPU加速サポート",
            "深層学習との統合",
            "変分推論と正規化フロー",
            "大規模データ対応",
            "分散計算サポート"
        ],
        "use_cases": "ベイズ深層学習、大規模推論"
    },
    "PyTorch/Pyro": {
        "description": "PyTorchベースの確率的プログラミング",
        "features": [
            "動的計算グラフ",
            "HMC/NUTSサポート",
            "変分推論",
            "ガイド付き推論",
            "研究向け柔軟性"
        ],
        "use_cases": "研究開発、カスタムモデル"
    },
    "emcee": {
        "description": "アンサンブルサンプラー専門ライブラリ",
        "features": [
            "Affine Invariant Ensemble Sampler",
            "高次元問題に効果的",
            "並列化サポート",
            "シンプルなAPI",
            "天体物理学で人気"
        ],
        "use_cases": "高次元パラメータ推定、物理学応用"
    },
    "scikit-learn": {
        "description": "機械学習ライブラリ（MCMC機能も含む）",
        "features": [
            "ベイズ推論手法",
            "ガウス過程",
            "混合モデル",
            "機械学習との統合",
            "豊富なドキュメント"
        ],
        "use_cases": "機械学習、データサイエンス"
    }
}

for lib_name, info in libraries_info.items():
    print(f"【{lib_name}】")
    print(f"概要: {info['description']}")
    print("主な機能:")
    for feature in info['features']:
        print(f"  • {feature}")
    print(f"適用分野: {info['use_cases']}")
    print()

# ライブラリ選択の指針
print("=== ライブラリ選択の指針 ===")
print()

selection_guide = {
    "初学者・一般的な用途": "PyMC - 直感的で豊富なドキュメント",
    "高性能・複雑なモデル": "Stan - 最適化された実装と豊富な診断",
    "深層学習との統合": "TensorFlow Probability, Pyro - GPU活用と大規模データ",
    "研究・カスタム実装": "Pyro, 自作実装 - 柔軟性と制御",
    "高次元パラメータ推定": "emcee - 効率的なアンサンブル手法",
    "機械学習応用": "scikit-learn - MLパイプラインとの統合",
    "教育・理解": "自作実装 - アルゴリズムの詳細理解"
}

for use_case, recommendation in selection_guide.items():
    print(f"{use_case}: {recommendation}")

print()
print("=== パフォーマンス比較の目安 ===")
print()

performance_comparison = {
    "実行速度": "Stan > TFP ≈ Pyro > PyMC > emcee > scikit-learn > 自作実装",
    "学習コスト": "PyMC < scikit-learn < emcee < TFP ≈ Pyro < Stan < 自作実装",
    "柔軟性": "自作実装 > Pyro ≈ TFP > Stan ≈ PyMC > emcee > scikit-learn",
    "診断機能": "Stan > PyMC > TFP ≈ Pyro > emcee > scikit-learn > 自作実装",
    "コミュニティ": "PyMC ≈ Stan > scikit-learn > TFP > Pyro > emcee > 自作実装"
}

for aspect, ranking in performance_comparison.items():
    print(f"{aspect}: {ranking}")

# 実装例のテンプレート（疑似コード）
print()
print("=== 各ライブラリの実装例（疑似コード） ===")
print()

print("【PyMC例】")
pymc_example = """
import pymc as pm

with pm.Model() as model:
    # 事前分布
    beta = pm.Normal('beta', mu=0, sigma=10, shape=p)
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # 尤度
    mu = pm.math.dot(X, beta)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
    
    # サンプリング
    trace = pm.sample(2000, tune=1000, cores=4)
"""
print(pymc_example)

print("【emcee例】")
emcee_example = """
import emcee

def log_prob(theta, x, y, yerr):
    # 対数確率密度の計算
    model = theta[0] * x + theta[1]
    return -0.5 * np.sum(((y - model) / yerr) ** 2)

# 初期位置の設定
ndim, nwalkers = 2, 50
pos = np.random.randn(nwalkers, ndim)

# サンプラーの作成と実行
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=(x, y, yerr))
sampler.run_mcmc(pos, 5000, progress=True)
"""
print(emcee_example)

print("【scikit-learn例（ベイズ推論）】")
sklearn_example = """
from sklearn.mixture import BayesianGaussianMixture
from sklearn.gaussian_process import GaussianProcessRegressor

# ベイズ混合モデル
bgm = BayesianGaussianMixture(n_components=3, random_state=42)
bgm.fit(X)
labels = bgm.predict(X)

# ガウス過程
gpr = GaussianProcessRegressor(random_state=42)
gpr.fit(X_train, y_train)
y_pred, y_std = gpr.predict(X_test, return_std=True)
"""
print(sklearn_example)

## 7.6 演習問題

### 問題1：適応的MCMCの改良
遅延棄却適応的メトロポリス（DRAM）を実装してください。棄却時に縮小した提案を試行する機能を追加します。

In [ ]:
# 問題1: DRAM（遅延棄却適応的メトロポリス）の実装
class DelayedRejectionAdaptiveMetropolis:
    """
    遅延棄却適応的メトロポリス法（DRAM）
    
    棄却時に縮小した提案を追加で試行する改良版適応的メトロポリス法
    
    実装のヒント：
    1. 通常の適応的メトロポリス法をベースとする
    2. 棄却時に、縮小ファクター（例：0.1）を適用した提案を試行
    3. 2段階の受理確率を計算
    4. 統計を適切に記録
    """
    
    def __init__(self, log_prob_fn, initial_cov_scale=0.1, adaptation_start=100, shrink_factor=0.1):
        self.log_prob_fn = log_prob_fn
        self.initial_cov_scale = initial_cov_scale
        self.adaptation_start = adaptation_start
        self.shrink_factor = shrink_factor  # 縮小ファクター
        
    def sample(self, initial_value, n_samples):
        """
        DRAMサンプリングの実行
        
        Returns:
        - samples: サンプル配列
        - acceptance_stats: {'first_stage': rate, 'second_stage': rate, 'total': rate}
        - final_cov: 最終共分散行列
        """
        # ここに実装してください
        # 
        # 実装手順：
        # 1. 適応的メトロポリス法の基本構造を作成
        # 2. 第1段階の提案が棄却された場合の処理を追加
        # 3. 第2段階（縮小提案）の受理確率計算を実装
        # 4. 統計の記録を適切に行う
        
        pass  # 学習者が実装

# テスト実行用のコード
# dram = DelayedRejectionAdaptiveMetropolis(
#     log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target)
# )
# 
# dram_samples, dram_stats, dram_cov = dram.sample(
#     initial_value=np.array([0.0, 0.0]),
#     n_samples=5000
# )
# 
# print(f"DRAM統計:")
# print(f"第1段階受理率: {dram_stats['first_stage']:.3f}")
# print(f"第2段階受理率: {dram_stats['second_stage']:.3f}") 
# print(f"総受理率: {dram_stats['total']:.3f}")

### 問題2：アンサンブルサンプラーの拡張
より多くのウォーカーを用いたアンサンブルサンプラーを実装し、並列化による高速化を実現してください。

In [ ]:
# 問題2: 並列化アンサンブルサンプラーの実装
from concurrent.futures import ThreadPoolExecutor
import threading

class ParallelEnsembleSampler:
    """
    並列化されたアフィン不変アンサンブルサンプラー
    
    実装のヒント：
    1. ウォーカーのグループを作成し、各グループを並列処理
    2. ThreadPoolExecutorまたはProcessPoolExecutorを使用
    3. ウォーカー間の同期を適切に管理
    4. メモリ効率を考慮した実装
    """
    
    def __init__(self, log_prob_fn, n_walkers, n_dim, n_threads=None):
        self.log_prob_fn = log_prob_fn
        self.n_walkers = n_walkers
        self.n_dim = n_dim
        self.n_threads = n_threads or min(4, n_walkers)
        
        if n_walkers < 2 * n_dim:
            raise ValueError(f"n_walkers ({n_walkers}) must be >= 2 * n_dim ({2 * n_dim})")
    
    def sample_parallel(self, initial_positions, n_steps):
        """
        並列アンサンブルサンプリングの実行
        
        実装課題：
        1. ウォーカーを複数のグループに分割
        2. 各グループを並列処理
        3. ステップごとの同期処理
        4. パフォーマンス測定
        
        Returns:
        - samples: サンプル配列 (n_steps, n_walkers, n_dim)
        - acceptance_rates: 各ウォーカーの受理率
        - timing_info: 実行時間情報
        """
        # ここに実装してください
        pass  # 学習者が実装
    
    def _worker_group_step(self, walker_indices, current_positions, current_log_probs):
        """
        ウォーカーグループの1ステップ実行（並列処理用）
        """
        # 並列処理用のワーカー関数を実装
        pass  # 学習者が実装

# 使用例（疑似コード）
# parallel_ensemble = ParallelEnsembleSampler(
#     log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
#     n_walkers=100,  # より多くのウォーカー
#     n_dim=2,
#     n_threads=4
# )
# 
# initial_pos = np.random.normal(0, 1, (100, 2))
# samples, rates, timing = parallel_ensemble.sample_parallel(initial_pos, 2000)
# 
# print(f"並列化効果:")
# print(f"実行時間: {timing['total_time']:.2f}秒")
# print(f"スレッド効率: {timing['thread_efficiency']:.2%}")
# print(f"平均受理率: {np.mean(rates):.3f}")

## まとめ

この章では、基本的なMCMC手法を超えた高度な手法について学習しました：

### 学習した手法

1. **適応的MCMC**：
   - サンプリング中のパラメータ自動調整
   - 経験共分散による提案分布最適化
   - 固定パラメータの限界を克服
   - ユーザー介入の削減

2. **アンサンブルサンプラー**：
   - 複数ウォーカーの協調サンプリング
   - アフィン不変性による頑健性
   - 高次元問題での優位性
   - 並列化による高速化

3. **並列・分散MCMC**：
   - 複数チェーンの並列実行
   - Gelman-Rubin診断による収束確認
   - スケーラビリティの向上
   - 計算資源の効率活用

4. **変分推論**：
   - 近似的だが高速な推論
   - ELBO最大化による最適化
   - MCMCの補完的手法
   - 大規模データへの対応

### 性能比較の結果

| 手法 | 精度 | 速度 | 並列性 | 実装難易度 | 適用範囲 |
|------|------|------|--------|------------|----------|
| Random Walk MH | 中 | 低 | 中 | 易 | 汎用 |
| Adaptive Metropolis | 高 | 中 | 中 | 中 | 汎用 |
| Ensemble Sampler | 高 | 中 | 高 | 中 | 高次元 |
| Parallel MCMC | 高 | 高 | 最高 | 難 | 大規模 |
| Variational Inference | 中 | 最高 | 高 | 難 | 近似OK |

### 実用的な指針

**手法選択の基準**：
- **高精度重視** → Adaptive Metropolis, Ensemble Sampler
- **高速計算重視** → Variational Inference
- **並列処理活用** → Parallel MCMC, Ensemble Sampler
- **高次元問題** → Ensemble Sampler, Adaptive methods
- **実装簡単さ** → Random Walk MH
- **大規模データ** → Variational Inference, Parallel MCMC

**ライブラリ活用**：
- 実際の応用では専用ライブラリの使用を強く推奨
- PyMC, Stan, emcee, TensorFlow Probabilityが主要選択肢
- 手法の理解のための自作実装も重要

### MCMCの発展と展望

**現在のトレンド**：
- **GPU加速**：TensorFlow Probability, PyTorchによる高速化
- **自動調整**：パラメータチューニングの自動化
- **深層学習統合**：ベイズニューラルネットワーク
- **近似手法**：変分推論や正規化フローとの融合

**将来の方向性**：
- **量子MCMC**：量子コンピュータを活用した新手法
- **適応的アーキテクチャ**：問題に応じた手法自動選択
- **分散システム**：クラウド環境での大規模並列処理
- **リアルタイム推論**：ストリーミングデータへの対応

### 学習のまとめ

MCMCは統計学・機械学習の基盤技術として：

1. **理論的基礎**：マルコフ連鎖理論と詳細釣り合い条件
2. **基本手法**：Metropolis-Hastings, Gibbs sampling
3. **高度な手法**：適応的MCMC, アンサンブルサンプラー, 並列MCMC
4. **実用技術**：収束診断, 効率評価, ライブラリ活用
5. **発展的手法**：変分推論との比較・統合

これらの知識を基に、実際の問題に適した手法を選択し、効率的なベイズ推論を実行できるようになることが目標です。

**次のステップ**：
- 実際のデータを用いた応用問題への挑戦
- 専門ライブラリの習得（PyMC, Stan等）
- 最新研究論文の追跡
- 自身の研究・業務での実践適用

MCMCは現在も活発に発展している分野であり、継続的な学習と実践を通じて、より深い理解と応用力を身につけることが重要です。